In [1]:
import scipy.io as sc
import numpy as np
from sklearn.svm import LinearSVC,SVC
import copy
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from HelperFunctions_ import classesInLabels

from SVMBDT import *

In [2]:
#Produce binary file from the training data 

def create_binary_file_for_svm_models(filname, svm_model):
    """

    Data structure: 
        svm_modus : uint8_t  [svm_linear, svm_rbf, svm_sparse] *Only svm_linear implemented yet 
        class_1 : uint8_t 
        class_2 : uint8_t 
        n_features (spectral bands) : uint8_t
        b coefficent : uint8_t 
        w weight array[] : floats
    """

    




    import struct 
    import numpy

    svm_modus = ["svm_linear", "svm_rbf", "svm_sparse"]

    class_1 = 0
    class_2 = 0

    try: 
        class_1 = svm.classes_[0]
        class_2 = svm.classes_[1]
    except: 
        print("svm.classes_ not defined")

    w_weights = svm_model.class_weigths
    n_features = len(w_weights)

In [9]:
#Make labels into land, water and clouds 
#water  = 2
#cropland = 3
#pasture = 4
#city = 5
#rock = 6
#thin_forest = 7
#thick_forest = 8 
#cloud = 9
#thin_clouds = 10

trondheim_new_labels = copy.deepcopy(new_trondheim_gt)

#trondheim_new_labels[trondheim_new_labels == 10] = 9
#trondheim_new_labels[trondheim_new_labels == 9] = 2

#nl = [4,5,6,7,8]

#for i in range(len(nl)):
#        trondheim_new_labels[trondheim_new_labels == nl[i]] = 3

In [4]:
trondheim_data = plt.imread("../Training_data/Trondheim_2022_08_23T10_26_43-bin3.png")
trondheim_gt = np.load("../Training_data/Trondheim_labels_2022-08-23.npz")["labels"]

In [5]:
#Preprocess the labeling 

new_trondheim_gt = []
y = trondheim_gt.flatten()

for i in range(0,len(y),3): 
    new_trondheim_gt.append(y[i])

new_trondheim_gt = np.array(new_trondheim_gt)

In [6]:
def rotate_image(image): 
    rotated_image = []
    for i in range(len(image[0])): 
        x_array = []
        for j in range(len(image)-1,-1,-1):
            x_array.append(image[j][i])
        rotated_image.append(x_array)

    return np.array(rotated_image)

In [7]:
#Make labels into land, water and clouds 
#water  = 2
#cropland = 3
#pasture = 4
#city = 5
#rock = 6
#thin_forest = 7
#thick_forest = 8 
#cloud = 9
#thin_clouds = 10

trondheim_new_labels = copy.deepcopy(new_trondheim_gt)


trondheim_new_labels[trondheim_new_labels == 10] = 9
#trondheim_new_labels[trondheim_new_labels == 9] = 3

nl = [4,5,6,7,8]

for i in range(len(nl)):
        trondheim_new_labels[trondheim_new_labels == nl[i]] = 3

classesInLabels(trondheim_new_labels)







array([2, 3, 9], dtype=int16)

In [11]:
new_trondheim = rotate_image(trondheim_data)

In [12]:
X = np.array(new_trondheim).reshape((-1,3))
X = preprocessing.scale(X, axis=0)
y = trondheim_new_labels.flatten()

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.95, random_state=1)

C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


In [301]:
tb = {0 : [[3,9],[2]], 1 : [[[3],[9]],[]]}
svm_tb = {0 : LinearSVC(), 1 : [LinearSVC(), False]}
svm_branch_models = SvmDesionTreeTrain(X_train, y_train, tb, svm_tb)

The training time is: 0.012 sec.


In [302]:
svm_00 = svm_branch_models[0]
svm_10 = svm_branch_models[1][0]

In [303]:
svm_10.classes_

array([3, 9], dtype=int16)

In [304]:
new_trondheim = rotate_image(trondheim_data)

In [305]:
plt.imshow(trondheim_new_labels.reshape(956,228))
plt.axis('off')
plt.savefig("Tronhdeim_few_labels", dpi = 300, bbox_inches="tight")
plt.close()

In [306]:
X = np.array(new_trondheim).reshape((-1,3))
X = preprocessing.scale(X, axis=0)
y = trondheim_new_labels.flatten()

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.95, random_state=1)

C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


In [307]:
svm = LinearSVC()
svm.fit(X_train, y_train)

C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [341]:
#Test pictures
"../Training_data/venice_2023-02-08_0944Z-bin3.png"

test_name = "venice_2023-02-08_0944Z-bin3.png"
test_data = plt.imread("../Training_data/" + test_name)

In [342]:
X_test = np.array(test_data).reshape((-1,3))
X_test = preprocessing.scale(X_test, axis=0)

C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(


In [368]:
prediction_array = []
X = copy.deepcopy(X_test)
w = svm_00.coef_[0]
b = 0.27 # 0.23

for i in range(len(X)): 
    pred = 0
    for j in range(len(X[0])): 
        pred += (X[i][j] * w[j] + b)
    if(np.mean(pred) >= 0): 
        prediction_array.append(1)
    else: 
        prediction_array.append(0)


In [369]:
svm_00.intercept_

array([-0.18004084])

In [379]:
pa = copy.deepcopy(prediction_array)

w = svm_10.coef_[0]
b = -0.1 #-0.17
count = 0


for i in range(len(pa)):
    pred = 0
    if(pa[i] == 1):
        for j in range(len(w)): 
            pred += (X[i][j] * w[j] + b)

        if(np.mean(pred) >= 0): 
            pa[i] = 2
            count += 1
        else: 
            pa[i] = 1

print(count)

42826


In [380]:
classesInLabels(pa)

array([0, 1, 2])

In [381]:
pa = np.array(pa)
plt.imshow(pa.reshape(len(test_data),len(test_data[0])))
plt.axis('off')
plt.savefig("../Z_Labeled_HYSPO_Data/" + test_name.split(".")[0] + "labeled", dpi = 300, bbox_inches="tight")
plt.close()

In [382]:
airport = sc.loadmat("../Training_data/abu-airport-1.mat")
print(airport)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Mar 23 18:23:02 2017', '__version__': '1.0', '__globals__': [], 'map': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8), 'data': array([[[ 641,  720,  947, ...,   43,   21,   25],
        [ 646,  721,  951, ...,   43,   20,   25],
        [ 677,  724,  954, ...,   40,   19,   24],
        ...,
        [ 825,  927, 1255, ...,   61,   24,   32],
        [ 846,  941, 1317, ...,   71,   34,   42],
        [ 836,  947, 1298, ...,   72,   32,   43]],

       [[ 624,  698,  945, ...,   44,   23,   26],
        [ 646,  699,  952, ...,   44,   21,   26],
        [ 637,  706,  936, ...,   43,   20,   24],
        ...,
        [ 855,  961, 1284, ...,   64,   29,   32],
        [ 852,  974, 1360, ...,   77,   35,   46],
        [ 818,  940, 1306, ...,   73,   

In [384]:
airport_data = airport["data"]
airport_gt = airport["map"]

In [387]:
len(airport_data[0][0])

205

In [406]:
X = np.array(airport_data).reshape((-1,205))
X = preprocessing.scale(X, axis=0)
y = airport_gt.flatten()

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.95, random_state=1)

In [407]:
svm = LinearSVC()
svm.fit(X_train, y_train)
predict = svm.predict(X)

C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [408]:
np.sum(predict == y)/sum(y)

68.80555555555556

In [402]:
pa = np.array(airport_data)
plt.imshow(airport_data.reshape(len(airport_data),len(airport_data[0])))
plt.axis('off')
plt.savefig("../Training_data/Airport_data" ,dpi = 300, bbox_inches="tight")
plt.close()

ValueError: cannot reshape array of size 2050000 into shape (100,100)

In [404]:
new_airport = []

for i in range(len(airport_data)):
    for j in range(len(airport_data[0])): 
        new_airport.append(np.mean(airport_data[i][j]))

new_airport = np.array(new_airport) 


In [405]:

plt.imshow(new_airport.reshape(len(airport_data),len(airport_data[0])))
plt.axis('off')
plt.savefig("../Training_data/Airport_data" ,dpi = 300, bbox_inches="tight")
plt.close()